In [7]:
from Classes.ArquivoInspecao import ArquivoInspecao
from Classes.BancoDados import BancoDados

In [ ]:
try:
    arquivo = ArquivoInspecao("Arquivos CSV\suicidios-total.csv")
    if arquivo.verifica:
        print("Arquivo CSV inspecionado com sucesso")
    else:
        print("O arquivo possui dados ausentes ou não possui dados")
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Erro ao inspecionar o arquivo: O arquivo não é um CSV!


In [9]:
testeDB = BancoDados('postgres', 'postgres', '123456', 'localhost', '5432')
testeDB.criacao_tabelas()

Conexão bem-sucedida!
Tabelas criadas com sucesso!
